# Data Processing


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


import numpy as np

data = pd.read_csv("MetroPT3.csv")
print(data.info())

# Data Introduction

The MetroPT-3 dataset was designed to facilitate the development of predictive maintenance, anomaly detection, and remaining useful life (RUL) prediction models for train compressors using deep learning and machine learning techniques. It comprises multivariate time series data collected from various analog and digital sensors installed on a train compressor. The data, recorded between February and August 2020, includes 15 signals such as pressures, motor current, oil temperature, and electrical signals from air intake valves. This dataset is suitable for incremental training and contains no sensitive information. Data preprocessing involves segmentation, normalization, and feature extraction. While the dataset itself is unlabeled, failure reports provided by the company are available to evaluate the performance of anomaly detection, failure prediction, and RUL estimation algorithms. Notably, the dataset does not contain any missing values.


the link : https://archive.ics.uci.edu/dataset/791/metropt+3+dataset

## Data Overview

In [ ]:
print(data.describe().round(2))
print(data.columns)

In [ ]:
data.head()

## Cleaning


In [ ]:
print(data.columns)
data = data.drop('Unnamed: 0', axis= 1)

In [ ]:

data.min()

## Convert the timestamp collumn into pandas.DateTime data type standarzation

In [ ]:
data.max()

In [ ]:
import datetime

#Check the current type of timestamp
print(f"Current type of timestamp is {type(data.timestamp[0])}")

#Convert timestamp to pandas.DateTime ISO 8601
data['timestamp'] = data['timestamp'].apply(pd.to_datetime, format = "%Y-%m-%d %H:%M:%S")

#Re-check the type
print(f"Current type of timestamp is {type(data.timestamp[0])}")

In [ ]:
print(data.head(10))

## Add a label feature

In [ ]:
#Create a new column for target variable called status, indicate the equipment has deficiencies and need to be maintained
# status = 0; system ups and running no No Failure
# status = 1; system downs and needs recovering ## Failure
labeled_data = data.copy()
labeled_data['status'] = 0
print(labeled_data.head(5))

In [ ]:
def to_datetime(xs):
  result = []
  format =  "%Y-%m-%d %H:%M:%S"
  for x in xs:
    result.append(pd.to_datetime(x, format = format))
  return result


failure_start_time = to_datetime(["2020-04-18 00:00:00", "2020-05-29 23:30:00", "2020-06-05 10:00:00", "2020-07-15 14:30:00"] )
failure_end_time   = to_datetime(["2020-04-18 23:59:00", "2020-05-30 06:00:00", "2020-06-07 14:30:00", "2020-07-15 19:00:00"] )

print(failure_start_time,"\n", failure_end_time[0].minute)

In [ ]:
def in_between(x, start, end):

  start_con = x >= start
  end_con = x<= end

  inbetween_con = start_con and end_con
  if inbetween_con:
    return 1
  else:
    return 0

In [ ]:
failure_indx = []
for i, (start_time, end_time) in enumerate(zip(failure_start_time, failure_end_time)):
  mask = labeled_data['timestamp'].apply(in_between, start = start_time, end = end_time)
  indx = labeled_data.index[mask == True].tolist()
  failure_indx += indx


print(f" Found {len(failure_indx)} samples representing failure state")

In [ ]:
#Set the sample with the timestamp falled between the failure time to 1
# labeled_data['status'].iloc[failure_indx] = 1
labeled_data.loc[failure_indx, 'status'] = 1
print(labeled_data['status'].value_counts())
print(f"We have {labeled_data['status'][labeled_data['status']==1].count()} positve samples" )

In [ ]:
print(f"Example of Failure state \n {labeled_data[labeled_data['status']==1].head()}")

## splite the dataset

In [ ]:
#Seperate Positive samples and Negative sample
pos_data = labeled_data[labeled_data['status'] == 1]
neg_data = labeled_data[labeled_data['status'] == 0]

#Print out the info of 2 dataset
print(f"Positive dataset\n {pos_data.info()}\n")
print(f"Negative dataset\n {neg_data.info()}\n")

As we can see, we have around 30K postive samples and 1500K negative sample. This indicates highly imbalanced dataset. Thus, we have to subsample the negative class to balance the training data. To achive this, we will randomly sample 30K negative sample from the set of 1500K sample.

In [ ]:
n_positives = int(pos_data['status'].count())
sub_neg_data = neg_data.sample(n_positives, random_state = 42)
print(f"Negative dataset after subsampling {sub_neg_data.info()}")

Now, we merge the postive set and negative set into one

In [ ]:
merged_data = pd.concat([pos_data, sub_neg_data], axis = 0)
print(f"Merged dataset\n")
merged_data.info()

## Valeurs abirrantes

In [ ]:
def investigate_outliers(data, c):
    q1 = data[c].quantile(0.25)
    q3 = data[c].quantile(0.75)
    iqr = q3 - q1
    ll = q1 - 1.5*iqr
    ul = q3 + 1.5*iqr

    num_outliers = data[data[c] < ll][c].count()  + data[data[c] > ul][c].count()
    if num_outliers>0:
        print(f"Found {num_outliers} oulier(s) for feature {c}")
    return {'col': c, 'n_outliers': num_outliers, 'll': ll, 'ul': ul, 'q1': q1, 'q3':q3}

print("\nDropping outliers ...\n")
clean_data = merged_data.copy()
for i in range(5):
  for c in clean_data.columns:
      if c not in ["Unnamed: 0","timestamp"]:
          cue = investigate_outliers(clean_data, c)
          if cue["n_outliers"] > 0 and (cue["q1"]!= cue["q3"]):
              print(f"Droping {cue['n_outliers']} from column {c}")
              clean_data = clean_data[clean_data[c]> cue["ll"]]
              clean_data = clean_data[clean_data[c]< cue["ul"]]
              print(f"{clean_data.shape[0]} samples left\n")
          elif (cue["q1"]== cue["q3"]):
              print("Skipping .. data has Q1 equals to Q3")
              print(f"{clean_data.shape[0]} rows left\n")


print("\nDropping Completed ...\n")
#Recheck data
for c in clean_data.columns:
    if c not in ["Unnamed: 0","timestamp","COMP", 'status']:
        cue = investigate_outliers(clean_data, c)

In [ ]:
#Investigate the columns with the binary values
binary_cols = ['LPS', 'Pressure_switch', 'Oil_level', 'Caudal_impulses']
#Ensure the the binary data is binary
clean_data[binary_cols] = clean_data[binary_cols].apply(np.round)

## III. Exploratory Data Analysis

### 1) Correlation

Describing the correlation between the features, the values closer to 1 or -1 represent a stronger relation.

In [ ]:
clean_data.corr().round(2)

We can see that our target variable "status" has high correlation with TP2, H1, DV_pressure, Oil_temparature, Motor_current, COMP, DV_electric and MPG.  

## Correlation

Below shows a Heat map,which can be used to analyse trends, from the below heat map you can see the trends in correlation of data.

In [ ]:
sns.heatmap(clean_data.corr(),annot=False )

In [ ]:
sns.pairplot(clean_data,  y_vars = ['status'] , plot_kws=  {'alpha' : 0.1})

Drawing box plot to find outliers, I plot it on scale data so it is easier to visualize different features' range.
As we can see our preprocessing function work perfectly that leaves no outliers


In [ ]:
clean_data.to_csv('RailGuadrs_Clean_Data.csv')
np.savez("RailGuadrs_Clean_Data.npz", clean_data.to_numpy())

# Models


## LinearRegression

In [ ]:
data_regression = pd.read_csv('RailGuadrs_Clean_Data.csv')
X = data_regression.iloc[:, 2:-1]
y = data_regression.iloc[:, -1]
X

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, X_test.shape)
X_train

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 

model_regression = LogisticRegression()
model_regression.fit(X_train_scaled, y_train)
print(f"Number of iterations: {model_regression.n_iter_}")

In [ ]:
print(X_train.columns)
print(f"the coef of the model are : {model_regression.coef_}")
print(f"the intercept is : {model_regression.intercept_}")

In [ ]:
X_test_scaled = scaler.transform(X_test)

In [ ]:
y_pred = model_regression.predict(X_test_scaled)

In [ ]:
confusion_matrix = ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.show()

In [ ]:
print(f'the accuracy is {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

In [ ]:
y_prob = model_regression.predict_proba(X_test)[:, 1]
for threshold in [0.2, 0.4, 0.6, 0.8]:
    y_pred_threshold = (y_prob > threshold).astype(int)
    fpr, tpr, _ = roc_curve(y_test, y_pred_threshold)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label='Threshold = {:.1f} (AUC = {:.8f})'.format(threshold, roc_auc))
    #print(threshold, format(roc_auc))
# Plot the "Random" line as a dashed line from (0,0) to (1,1) for reference
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')

# Label axes and set the title for the plot
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR) or Sensitivity')
plt.title('Receiver Operating Characteristic (ROC) Curve with Different Thresholds')
plt.legend(loc='lower right')
plt.show()

## Predict with LogisticRegression

In [ ]:
# Example: new sample based on the features
new_sample = {
    'timestamp': [193.293044], 
    'TP2': [-0.018],
    'TP3': [8.248],
    'H1': [8.238],
    'DV_pressure': [-0.024],
    'Reservoirs': [8.248],
    'Oil_temperature': [49.450],
    'Motor_current': [0.0400],
    'COMP': [1.0],
    'DV_eletric': [0.0],
    'Towers': [1.0],
    'MPG': [1.0],
    'LPS': [0.0],
    'Pressure_switch': [1.0],
    'Oil_level': [1.0],
    'Caudal_impulses': [1.0]
}

new_df = pd.DataFrame(new_sample)
new_df

In [ ]:
training_feature_names = X_train.columns
new_df = new_df[training_feature_names]

In [ ]:
new_scaled = scaler.transform(new_df)

# Predict the class and probability
y_pred = model_regression.predict(new_scaled)
y_pred_prob = model_regression.predict_proba(new_scaled)[:, 1]

print(f"Prediction (Class): {y_pred}")
print(f"Probability of class 1: {y_pred_prob}")

# Deploy the Model

In [ ]:
import joblib

# Save the trained logistic regression model
joblib.dump(model_regression, 'logistic_regression_model.pkl')

# Save the scaler
joblib.dump(scaler, 'scaler_logistic.pkl')


# Naive Bayes


In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB, BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv('RailGuadrs_Clean_Data.csv')


# Features and target
features = ['timestamp', 'TP2', 'TP3', 'H1', 'DV_pressure', 'Reservoirs',
            'Oil_temperature', 'Motor_current', 'COMP', 'DV_eletric',
            'Towers', 'MPG', 'LPS', 'Pressure_switch', 'Oil_level', 'Caudal_impulses']
target = 'status'


In [ ]:
# Convert timestamp to a scaled numeric value
dataset['timestamp'] = pd.to_datetime(dataset['timestamp'], errors='coerce')
dataset['timestamp'] = (dataset['timestamp'] - dataset['timestamp'].min()) / np.timedelta64(1, 'D')


In [ ]:
X = dataset[features]
y = dataset[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize classifiers
gaussian_classifier = GaussianNB()
bernoulli_classifier = BernoulliNB()

In [ ]:
# MultinomialNB requires non-negative features, use MinMaxScaler for preprocessing
multinomial_classifier = Pipeline([
    ('Normalize', MinMaxScaler()),
    ('MultinomialNB', MultinomialNB())
])

In [ ]:
# Train the models
gaussian_classifier.fit(X_train, y_train)
multinomial_classifier.fit(X_train, y_train)
bernoulli_classifier.fit(X_train, y_train)

In [ ]:
# Evaluate each model
for name, model in [
    ("GaussianNB", gaussian_classifier),
    ("MultinomialNB", multinomial_classifier),
    ("BernoulliNB", bernoulli_classifier)
]:
    y_pred = model.predict(X_test)
    print(f"\n{name} Classification Report:")
    print(classification_report(y_test, y_pred))
    ConfusionMatrixDisplay.from_predictions(y_test, y_pred).plot()

In [ ]:
new_sample = {
    'timestamp': [1.5],  # Replace with a realistic scaled timestamp
    'TP2': [-0.018],
    'TP3': [8.248],
    'H1': [8.238],
    'DV_pressure': [-0.024],
    'Reservoirs': [8.248],
    'Oil_temperature': [49.450],
    'Motor_current': [0.0400],
    'COMP': [1.0],
    'DV_eletric': [0.0],
    'Towers': [1.0],
    'MPG': [1.0],
    'LPS': [0.0],
    'Pressure_switch': [1.0],
    'Oil_level': [1.0],
    'Caudal_impulses': [1.0]
}

In [ ]:
# Create DataFrame
new_df = pd.DataFrame(new_sample)

# Ensure column order matches training data
training_feature_names = [
    'timestamp', 'TP2', 'TP3', 'H1', 'DV_pressure', 'Reservoirs',
    'Oil_temperature', 'Motor_current', 'COMP', 'DV_eletric',
    'Towers', 'MPG', 'LPS', 'Pressure_switch', 'Oil_level', 'Caudal_impulses'
]
new_df = new_df[training_feature_names]

In [ ]:
# Predict the class and probability for the new sample
y_pred_gaussian = gaussian_classifier.predict(new_df)
y_pred_gaussian_prob = gaussian_classifier.predict_proba(new_df)[:, 1]

# Map the prediction to class labels
class_labels = {0: "Not Failure", 1: "Failure"}
predicted_label = [class_labels[label] for label in y_pred_gaussian]

# Output the results
print(f"Prediction (Class): {predicted_label}")
print(f"Probability of Failure: {y_pred_gaussian_prob}")


## Deploy the Naive bayes

In [ ]:
joblib.dump(gaussian_classifier, 'gaussian_nb_model.pkl')
joblib.dump(multinomial_classifier, 'multinomial_nb_model.pkl')
joblib.dump(bernoulli_classifier, 'bernoulli_nb_model.pkl')

# XGBoost

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.preprocessing import MinMaxScaler


In [ ]:
# Load dataset
df = pd.read_csv('RailGuadrs_Clean_Data.csv')

# Convert timestamp to numerical values
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df['timestamp'] = (df['timestamp'] - df['timestamp'].min()) / pd.Timedelta(days=1)

# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 0'])

# Display the first rows of the cleaned dataset
print(df.head())


In [ ]:
# Features and target
X = df.drop(columns=['status'])  # All columns except 'status'
y = df['status']  # Target column


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)

# Normalize features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Initialize XGBoost model
xgb_model = XGBClassifier(
    max_depth=6,
    n_estimators=100,
    learning_rate=0.1,
    objective='binary:logistic',
    random_state=42
)

# Train the model
xgb_model.fit(X_train, y_train)

print("Model training completed!")


In [ ]:
y_pred = xgb_model.predict(X_test)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Display confusion matrix
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)


In [ ]:
import joblib

# Save the model
joblib.dump(xgb_model, 'xgb_model.pkl')

# Save the scaler
joblib.dump(scaler, 'scaler_xgb.pkl')

print("Model and scaler saved successfully!")

In [ ]:
# Load the saved model
xgb_model = joblib.load('xgb_model.pkl')
scaler = joblib.load('scaler_xgb.pkl')


# Example new data
new_sample = {
    'timestamp': [1.5],  # Replace with a realistic scaled timestamp
    'TP2': [30.018],
    'TP3': [8.248],
    'H1': [25.238],
    'DV_pressure': [5.024],
    'Reservoirs': [-88.248],
    'Oil_temperature': [80.450],
    'Motor_current': [10.0400],
    'COMP': [0.0],
    'DV_eletric': [0.0],
    'Towers': [0.0],
    'MPG': [0.0],
    'LPS': [1.0],
    'Pressure_switch': [0.0],
    'Oil_level': [1.0],
    'Caudal_impulses': [1.0]
}


# Convert to DataFrame
new_df = pd.DataFrame(new_sample)

# Scale the new data
new_df_scaled = scaler.transform(new_df)

# Predict
prediction = xgb_model.predict(new_df_scaled)
print("Prediction (Class):", "Failure" if prediction[0] == 1 else "Normal")


# DeepLearning

##  dataset Preparation

In [ ]:
# Load dataset
df = pd.read_csv('RailGuadrs_Clean_Data.csv')

# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 0', 'timestamp'])

# Drop any remaining NaN values
df = df.dropna()

# Features and target
X = df.drop(columns=['status'])  # All columns except 'status'
y = df['status']  # Target column

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam



# Load dataset
df = pd.read_csv('RailGuadrs_Clean_Data.csv')

# Convert timestamp to numerical values
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df['timestamp'] = (df['timestamp'] - df['timestamp'].min()) / pd.Timedelta(days=1)

# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 0'])

# Features and target
X = df.drop(columns=['status'])
y = df['status']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
# Load dataset
df = pd.read_csv('RailGuadrs_Clean_Data.csv')

# Convert timestamp to numerical values
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df['timestamp'] = (df['timestamp'] - df['timestamp'].min()) / pd.Timedelta(days=1)

# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 0'])

# Features and target
X = df.drop(columns=['status'])
y = df['status']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [ ]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

# Predict on the test set
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)


In [ ]:
# Save the model
model.save('neural_net_model.h5')

# Save the scaler
import joblib
joblib.dump(scaler, 'scaler_net.pkl')

print("Model and scaler saved successfully!")
